# Final Project: Honeycomb AKLT as PEPS

Submitted by Yoav Zack, ID 211677398

## `julia` Implementation

We will implement those two wave functions as PEPS in `julia` using `TensorOperations`:

In [ ]:
using LinearAlgebra, TensorOperations, Plots, Printf, BenchmarkTools

First, we define the Honeycomb structure, and some useful functions on it:

In [ ]:
struct HoneycombLattice
    shape::String
    a::Real
    verts::Vector{Tuple{Int64, Tuple}}
    bonds::Vector{Tuple}
    ψ::Vector{Array{Float64}}

    function HoneycombLattice(shape::String, a::Real, shift::Tuple, n::Integer, bc::String)
        @assert shape ∈ ("pyramid", "flower", "bricks") "Invalid lattice shape"
        @assert a > 0 "Bond length `a` must be positive"
        @assert length(shift) == 2 "Lattice shift must be of length 2"
        @assert n > 0 "Lattice order `n` must be positive"
        @assert bc ∈ ("periodic", "open") "Invalid lattice boundary condition"

        # define verts location based on lattice shape
        if shape == "pyramid"
            dx = a * sin(π/6)
            dy = a * cos(π/6)
            
            locs = [(0.0, 0.0)]
            for (i,x) in enumerate(range(start=dx, length=n, step=a+dx))
                for y in range(-dy*i, stop=dy*i, length=i+1)
                    append!(locs, [(-x,y), (-x-a,y)])
                end
            end
        
            for (j, y) in enumerate(range(start=-dy*(n-1), stop=dy*(n-1), length=n))
                append!(locs, [(-n*(dx+a)-dx, y)])
            end
        elseif shape == "bricks"
            if n == 1
                locs = [(0.0, 0.0), (0.0, a), (2*a, 0.0), (a, a), (a, 0.0), (2*a, a)]
            else
                locs = [(0.0, 0.0), (0.0, a), (a, 2*a), (a, a)]
                for i in 1:n
                    if i%2==0
                        append!(locs, [r.+(i-1).*(a,0) for r in [(a,2*a),(2*a,2*a),(2*a,a)]])
                    else
                        append!(locs, [r.+(i-1).*(a,0) for r in [(a,0),(2*a,0),(2*a,a)]])
                    end
                end
            end
            
        elseif shape == "flower"
            
        end
            
        # add shift to vertex locations and define verts array
        verts = [(ind, loc .+ shift) for (ind, loc) in enumerate(locs)]
        
        # detect bonds
        bonds = detectbonds(verts, a, shape)

        # fill in tensors
        A = AKLT_PEPS()
        missing = detectmissing(verts, bonds)
        ψ = []
        for (vert, miss) in zip(verts, missing)
            (ind, loc) = vert
            if length(miss) == 0
                push!(ψ, A)
            elseif length(miss) == 1
                push!(ψ, A[:,:,:,1])
            elseif length(miss) == 2
                push!(ψ, A[:,:,1,1])
            elseif length(miss) == 3
                push!(ψ, A[:,1,1,1])
            else
                error("Invalid number of missing bonds from vertex ", ind)
            end
        end
        
        # create structure
        new(shape, a, verts, bonds, ψ)
    end
end

function detectbonds(verts::Vector{Tuple{Int64, Tuple{Float64, Float64}}}, a::Real, shape::String=nothing)
    bonds = []
    bond_count = ones(Int64, length(verts))
    for vert in verts
        (ind, loc) = vert
        for (ind_j, loc_j) in nearestNeighboors(verts[ind:end], vert, a)
            if shape =="bricks" && (ind_j == ind + 1 && ind % 3 == 1 && ind != 1)
                continue
            end
            push!(bonds, ((ind, bond_count[ind]), (ind_j, bond_count[ind_j])))
            bond_count[ind] += 1
            bond_count[ind_j] += 1
        end
    end
    return bonds
end

function countneighbors(verts::Vector{Tuple{Int64, Tuple{Float64, Float64}}}, vert::Tuple{Int64, Tuple{Float64, Float64}}, bonds::Vector{Any})
    c = 0
    vert_bonds = []
    (ind, loc) = vert
    for bond in bonds
        if ind ∈ bond
            c += 1
            push!(vert_bonds, bond)
        end
    end
    return c, vert_bonds
end

function nearestNeighboors(verts::Vector{Tuple{Int64, Tuple{Float64, Float64}}}, vert::Tuple{Int64, Tuple{Float64, Float64}}, a::Real)
    neighbors = []
    (ind, loc) = vert
    for (ind_i, loc_i) in verts
        d = sum((loc .- loc_i).^2)
        if abs(d - a) < 1e-1*a
            push!(neighbors, (ind_i, loc_i))
        end
    end
    return neighbors
end

function detectmissing(verts::Vector{Tuple{Int64, Tuple{Float64, Float64}}}, bonds::Vector{Any})
    missing = [[1,2,3] for i in 1:length(verts)]
    for bond in bonds
        for (vert_ind, bond_ind) in bond
            deleteat!(missing[vert_ind], findall(x->x==bond_ind,missing[vert_ind]))
        end
    end
    return missing
end

function showlattice(fig::Plots.Plot, lat::HoneycombLattice, dolabel::Bool)
    x = [loc[1] for (ind, loc) in lat.verts]
    y = [loc[2] for (ind, loc) in lat.verts]
    for bond in lat.bonds
        inds = [b[1] for b in bond]
        dx = [lat.verts[i][2][1] for i in inds]
        dy = [lat.verts[i][2][2] for i in inds]
        plot!(fig, dx, dy, label="", color="blue")
    end
    scatter!(fig, x, y, aspect_ratio=1, label="", color="red")

    if dolabel
        for (ind, loc) in lat.verts
            annotate!(fig, loc[1], loc[2], ind)
        end
    end
    return fig
end

The tensors are defined using the following function:

In [ ]:
function AKLT_PEPS()
    Θ = zeros(4,2,2,2)
    Σ = zeros(2,2)
    A = zeros(4,2,2,2)
    
    # define Σ the singlet matrix
    Σ[1,2] = 1/sqrt(2); Σ[2,1] = -1/sqrt(2);
    
    # define Θ the projection operator
    for a in 1:2
        for b in 1:2
            for c in 1:2
                Θ[1,a,b,c] = (a+b+c == 3)
                Θ[2,a,b,c] = (a+b+c == 4)/sqrt(3)
                Θ[3,a,b,c] = (a+b+c == 5)/sqrt(3)
                Θ[4,a,b,c] = (a+b+c == 6)
            end
        end
    end
    
    # define A the total tensor
    for σ in 1:4
        for b in 1:2
            A[σ,:,b,:] = Θ[σ,:,b,:]*Σ*sqrt(6/10)
        end
    end
    return A
end

For example, here are two possible lattices:

In [ ]:
lat = HoneycombLattice("bricks", 1, (5.0,1.0), 3, "open")
fig = scatter()
fig = showlattice(fig, lat, true)

In [ ]:
lat = HoneycombLattice("pyramid", 1, (5.0,1.0), 2, "open")
fig = scatter()
fig = showlattice(fig, lat, true)

Next, in order to calculate local expectation values of a specific site, we need to orthogonalize the PEPS onto the site. This is done by contracting bonds of sites and then split them again using SVD, and is the reason for the more complicated bond data-structure in the `HoneycombLattice` struct. We start by writing a function to normalize the PEPS:

In [ ]:
function bondtensor(lat::HoneycombLattice, ind_i::Int64, ind_j::Int64)
    @assert ind_i >= 1 && ind_i <= length(ψ) "Bond 1 out of range"
    @assert ind_j >= 1 && ind_j <= length(ψ) "Bond 2 out of range"
    @assert ind_i != ind_j "Bond indices cannot be equal"

    # order bond indices
    if ind_i > ind_j
        (ind_i, ind_j) = (ind_j, ind_i)
    end

    # fond bond in list of bonds
    found = false
    for bond in bonds
        ind_bond = (bond[1][1], bond[2][1])
        if (ind_i, ind_j) == ind_bond
            found = true
            break
        end
    end
    @assert found==true "Bond does not appear in lattice"
    
    # contract together sites ind_i and ind_j according to their bond indices
    if bond[1][2] == 1 && bond[2][2] == 1
        @tensor ψ_bond[σ,a,b,c,d] = lat.ψ[ind_i][σ,l,a,b] * lat.ψ[ind_j][σ,l,c,d]
    elseif bond[1][2] == 2 && bond[2][2] == 1
        @tensor ψ_bond[σ,a,b,c,d] = lat.ψ[ind_i][σ,a,l,b] * lat.ψ[ind_j][σ,l,c,d]
    elseif bond[1][2] == 3 && bond[2][2] == 1
        @tensor ψ_bond[σ,a,b,c,d] = lat.ψ[ind_i][σ,a,b,l] * lat.ψ[ind_j][σ,l,c,d]
    elseif bond[1][2] == 1 && bond[2][2] == 2
        @tensor ψ_bond[σ,a,b,c,d] = lat.ψ[ind_i][σ,l,a,b] * lat.ψ[ind_j][σ,c,l,d]
    elseif bond[1][2] == 2 && bond[2][2] == 2
        @tensor ψ_bond[σ,a,b,c,d] = lat.ψ[ind_i][σ,a,l,b] * lat.ψ[ind_j][σ,c,l,d]
    elseif bond[1][2] == 3 && bond[2][2] == 2
        @tensor ψ_bond[σ,a,b,c,d] = lat.ψ[ind_i][σ,a,b,l] * lat.ψ[ind_j][σ,c,l,d]
    elseif bond[1][2] == 1 && bond[2][2] == 3
        @tensor ψ_bond[σ,a,b,c,d] = lat.ψ[ind_i][σ,l,a,b] * lat.ψ[ind_j][σ,c,d,l]
    elseif bond[1][2] == 2 && bond[2][2] == 3
        @tensor ψ_bond[σ,a,b,c,d] = lat.ψ[ind_i][σ,a,l,b] * lat.ψ[ind_j][σ,c,d,l]
    elseif bond[1][2] == 3 && bond[2][2] == 3
        @tensor ψ_bond[σ,a,b,c,d] = lat.ψ[ind_i][σ,a,b,l] * lat.ψ[ind_j][σ,c,d,l]
    end

    return ψ_bond
end

function SvdBondTensor_AKLT!(ψ::Vector{Array{Float64}}, b::Int64, ψ_bond::Array{Float64}, ortho_left=true, maxM=-1, renorm=false)
    """
    SvdBondTensor_AKLT!(ψ::Vector{Array{Float64}}, b::Int64, ψ_bond::Array{Float64}, ortho_left=true, maxM=-1, renorm=false)

    Update the Matrix Product State (MPS) by performing a Singular Value Decomposition (SVD) on the bond tensor.

    # Arguments
    - `ψ`: Array of matrices representing the MPS.
    - `b`: Index of the bond between adjacent sites (1 to length(psi)-1).
    - `ψ_bond`: Bond tensor connecting sites `b` and `b+1`.
    - `ortho_left`: If true, orthogonally update the matrix at site `b` (left-canonical). If false, orthogonally update the matrix at site `b+1` (right-canonical).
    - `maxM`: The maximum number of singular values to keep in each step. If -1, no truncation is performed.
    - `renorm`: If true, renormalize the singular values after the SVD.

    # Returns
    - `trunc`: Truncation error if truncation occurs, otherwise 0.

    """
    @assert length(size(ψ_bond)) == 4 "psi_bond is expected to be a rank-4 tensor"

    trunc = 0.

    M0 = size(ψ_bond)[1]
    M2 = size(ψ_bond)[4]

    # Reshape the bond tensor into a matrix for SVD
    ψ_bond_matrix = reshape(ψ_bond, (M0*3, M2*3))

    # Perform SVD on the reshaped bond tensor matrix
    U, S, V = svd(ψ_bond_matrix)

    # Optionally renormalize the singular values
    if renorm
        S /= norm(S)
    end

    # Find the last non-zero singular value
    M1 = findlast(S .> 1e-16)

    # Retain only relevant parts of U, S, and V
    U, S, V = U[:, 1:M1], S[1:M1], V[:, 1:M1]

    # If 0 < maxM < M1 - truncation, perform truncation
    if maxM > 0 && M1 > maxM
        trunc = sum(S[maxM:end].^2)
        U, S, V = U[:, 1:maxM], S[1:maxM], V[:, 1:maxM]
        S /= sqrt(1. - trunc)
        M1 = maxM
    end

    # Update the MPS matrices based on the SVD results
    if ortho_left
        ψ[b] = reshape(U, (M0, 3, M1))
        ψ[b+1] = reshape(Diagonal(S) * V', (M1, 3, M2))
    else
        ψ[b] = reshape(U * Diagonal(S), (M0, 3, M1))
        ψ[b+1] = reshape(V', (M1, 3, M2))
    end

    return trunc
end

function NormalizePsi_AKLT!(ψ::Vector{Array{Float64}})
    """
    NormalizePsi_AKLT(ψ::Vector{Array{Float64}})

    Normalize and bring the orthogonality center to the first site in a Matrix Product State (MPS), assuming right canonical form.

    # Arguments
    - `ψ`: Array of matrices representing the MPS.

    """
    L = length(ψ)

    # Iterate from the last site to the second site
    for i in L-1:-1:1
        # Calculate the bond tensor between current site i and i+1
        ψ_bond = BondTensor_AKLT(ψ, i)

        # Orthogonally update the MPS using the Singular Value Decomposition (SVD), assuming right canonical form
        SvdBondTensor_AKLT!(ψ, i, ψ_bond, false, -1, true)
        # ortho_left = false => Update the matrix at site i+1 (right canonical form)
        # maxM = -1 => No truncation
        # renorm = true => Renormalize the singular values
    end
end